In [11]:
import pandas as pd
import yfinance as yf
import numpy as np

In [12]:
start_date = '2023-09-01'
code = '^NSEBANK'
df=yf.download(code,start=start_date)
df=df.drop(['Adj Close','Volume'],axis=1)

[*********************100%***********************]  1 of 1 completed


In [13]:
#Swing Lows
support=df
support=round(support,2)
support.index=pd.to_datetime(support.index).date
support=support[support.Close==support.Close.rolling(5,center=True).min()]
support['Swing']='SL'
#support['Swing']=np.where(support.Low<support.Low.shift(1),'LL','HL')
support.tail()

,Open,High,Low,Close,Swing
2023-11-01,42694.25,42815.15,42589.65,42700.95,SL
2023-11-17,43656.25,43872.85,43513.85,43583.95,SL
2023-11-22,43663.60,43692.50,43230.95,43449.60,SL
2023-12-13,47190.75,47201.95,46886.90,47092.25,SL
2023-12-20,48085.35,48166.15,47202.65,47445.30,SL


In [14]:
#Swing Highs
resistance=df
resistance=round(resistance,2)
resistance.index=pd.to_datetime(resistance.index).date
resistance=resistance[resistance.Close==resistance.Close.rolling(5,center=True).max()]
resistance['Swing']='SH'
#support['Swing']=np.where(support.Low<support.Low.shift(1),'LL','HL')
resistance.tail()

,Open,High,Low,Close,Swing
2023-11-21,43790.95,43790.95,43586.45,43689.15,SH
2023-12-05,46895.90,47230.55,46653.65,47012.25,SH
2023-12-11,47233.30,47588.00,47186.60,47314.25,SH
2023-12-15,47939.90,48219.95,47632.85,48143.55,SH
2023-12-28,48479.05,48636.45,48343.15,48508.55,SH


In [15]:
temp=pd.concat([support,resistance])
temp

,Open,High,Low,Close,Swing
2023-09-06,44494.65,44577.00,44207.25,44409.10,SL
2023-09-22,44707.35,44996.75,44548.90,44612.05,SL
2023-09-28,44700.35,44756.80,44248.20,44300.95,SL
2023-10-04,44108.15,44161.35,43857.50,43964.05,SL
2023-10-09,44057.80,44113.20,43796.75,43886.50,SL
2023-10-26,42708.40,42716.20,42105.40,42280.15,SL
2023-11-01,42694.25,42815.15,42589.65,42700.95,SL
2023-11-17,43656.25,43872.85,43513.85,43583.95,SL
2023-11-22,43663.60,43692.50,43230.95,43449.60,SL
2023-12-13,47190.75,47201.95,46886.90,47092.25,SL


In [16]:
dt=yf.download(code,start=start_date)
dt=round(dt,2)
dt.index=pd.to_datetime(dt.index).date

[*********************100%***********************]  1 of 1 completed


In [17]:
dt = dt.merge(temp[['Swing']], left_index=True, right_index=True, how='left')

In [18]:
dt=dt.drop(['Adj Close','Volume'],axis=1)

In [19]:
dt.tail()

,Open,High,Low,Close,Swing
2023-12-26,47576.40,47838.45,47411.65,47724.85,NaN
2023-12-27,47818.50,48347.65,47806.70,48282.20,NaN
2023-12-28,48479.05,48636.45,48343.15,48508.55,SH
2023-12-29,48374.85,48477.70,48091.85,48292.25,NaN
2024-01-01,48203.45,48450.00,48044.15,48234.30,NaN


In [20]:
#Stop Loss
def long_stop(long_price,system_low):
    system_points=long_price - system_low
    
    if  long_price*0.02 < system_points:
        stop_price=round(long_price*0.98,2)
    else:
        stop_price=system_low
    
    return stop_price

def short_stop(short_price,system_high):
    system_points=system_high - short_price
    
    if short_price*0.02 < system_points:
        stop_price=round(short_price*1.02,2)
    else:
        stop_price=system_high 
    
    return stop_price

In [22]:
long_price=0
long_date=0
long_entry_price=0
long_stop_price=0
system_low=0
long_position=0
long_exit_date=0
long_exit_price=0
lp=0                            #Copy of long_price
sl=0                            #Copy of system_low
SL_cnt=0 

long_trades=[]
long_trail=[]

short_price=0
short_date=0
short_entry_price=0
short_stop_price=0
system_high=0
short_position=0
short_exit_date=0
short_exit_price=0
sp=0                            #Copy of short_price
sh=0                            #Copy of system_high
SH_cnt=0  

short_trades=[]
short_trail=[]

long_position=None
short_position=None

for i in range(len(dt)):
    
    #Long Entry Point
    if dt.Swing.iloc[i] == 'SH':
        long_price = dt.Close.iloc[i]
        system_low = dt.Close.iloc[i]
        
    #System Low    
    if dt.Swing.iloc[i] != 'SH' and long_price != 0:
        
        if dt.Close.iloc[i] < system_low:
            system_low=dt.Close.iloc[i]
            
    if dt.Swing.iloc[i] == 'SL' and dt.Close.iloc[i] > system_low:
        system_low=dt.Close.iloc[i]
    
    #Short Entry Point
    if dt.Swing.iloc[i] == 'SL':
        short_price = dt.Close.iloc[i]
        system_high = dt.Close.iloc[i]
        #print(short_price)
        #print(i)
    
    #System High 
    if dt.Swing.iloc[i] != 'SL' and short_price != 0:
        
        if dt.Close.iloc[i] > system_high:
            system_high = dt.Close.iloc[i]
            #print(system_high)
            
    if dt.Swing.iloc[i] == 'SH' and dt.Close.iloc[i] < system_high:
        system_high=dt.Close.iloc[i]
            
    #Long Entry Trigger
    if dt.Swing.iloc[i] != 'SH' and long_price!=0 and long_position==None:
        
        if dt.Open.iloc[i] > long_price and dt.Close.iloc[i] > long_price:
            long_entry_price=dt.Open.iloc[i]
            #print(long_entry_price)
            long_date=dt.index[i]
            #print(long_date)
            long_stop_price=long_stop(long_price,system_low)
            sl=system_low
            lp=long_price
            long_position=True
        elif dt.Close.iloc[i] > long_price:
            long_entry_price=dt.Close.iloc[i]
            #print(long_entry_price)
            long_date=dt.index[i]
            #print(long_date)
            long_stop_price=long_stop(long_price,system_low)
            sl=system_low
            lp=long_price
            long_position=True
        
    #Long Stops
    if long_position == True:
        if dt.Open.iloc[i] <= sl*0.98:
            long_exit_date=dt.index[i]
            long_exit_price=dt.Open.iloc[i]
        elif dt.Close.iloc[i] <= long_stop_price:
            long_exit_date=dt.index[i]
            long_exit_price=long_stop_price
        elif dt.Swing.iloc[i] == 'SL':
            long_stop_price=dt.Close.iloc[i]
            SL_cnt=SL_cnt+1
        elif dt.Close.iloc[i] >= lp*1.02 and SL_cnt==0:
            long_stop_price=long_entry_price
            
        #Exiting the long trade     
        if i==len(dt)-2:
            long_exit_date=dt.index[i]
            long_exit_price=dt.Close.iloc[i]
            
        long_trail.append({'Date':dt.index[i],'Long Stop':long_stop_price})
    
        
    #Long Trade Records    
    if long_exit_price !=0:
        long_trades.append({'Trade direction':'Long',
                        'Entry Date':long_date,
                       'Long Entry Price':long_entry_price,
                       'Stop': long_stop_price,
                      'Exit Price': long_exit_price,
                      'Profit/Loss': long_exit_price - long_entry_price,
                      'Exit Date': long_exit_date})    
        
        long_exit_price=0
        long_date=0
        long_entry_price=0
        long_stop_price=0
        long_exit_date=0
        long_position=None
        SL_cnt=0      
    
    #Short Entry Trigger
    if dt.Swing.iloc[i] != 'SL' and short_price != 0 and short_position == None:
              
        if dt.Open.iloc[i] < short_price and dt.Close.iloc[i] < short_price:
            short_entry_price=dt.Open.iloc[i]
            short_date=dt.index[i]
            short_stop_price=short_stop(short_price,system_high)
            sh=system_high
            sp=short_price
            short_position=True
        elif dt.Close.iloc[i] < short_price:
            short_entry_price=dt.Close.iloc[i]
            short_date = dt.index[i]
            short_stop_price=short_stop(short_price,system_high)
            sh=system_high
            sp=short_price
            short_position=True
            
    #Short Stops
    if short_position == True:
        if dt.Open.iloc[i] >= sh*1.02:
            short_exit_date=dt.index[i]
            short_exit_price=dt.Open.iloc[i]
        elif dt.Close.iloc[i] >=  short_stop_price:
            short_exit_date=dt.index[i]
            short_exit_price=short_stop_price
        elif dt.Swing.iloc[i] == 'SH':
            short_stop_price = dt.Close.iloc[i]
            SH_cnt+=1
        elif dt.Close.iloc[i] <= sp*0.98 and SH_cnt==0:
            short_stop_price=short_entry_price
                   
        short_trail.append({'Date':dt.index[i],'Short Stop':short_stop_price})  
        
    #Short Trade Records 
    if short_exit_price !=0:
        short_trades.append({'Trade direction':'Short',
                        'Entry Date':short_date,
                       'Short Entry Price':short_entry_price,
                       'Stop': short_stop_price,
                      'Exit Price': short_exit_price,
                      'Profit/Loss': short_entry_price - short_exit_price,
                      'Exit Date': short_exit_date})    
        
        short_exit_price=0
        short_date=0
        short_entry_price=0
        short_stop_price=0
        short_exit_date=0
        short_position=None
        SH_cnt=0

In [23]:
short_trades_df=pd.DataFrame(short_trades)
short_trail_df = pd.DataFrame(short_trail) 

short_trail_df.set_index('Date', inplace=True)
short_trades_df.set_index('Entry Date', inplace=True)

In [24]:
short_trades_df

,Trade direction,Short Entry Price,Stop,Exit Price,Profit/Loss,Exit Date
Entry Date,,,,,,
2023-09-27,Short,44465.90,44360.60,44360.60,105.3,2023-10-11
2023-10-19,Short,43616.75,43039.15,43039.15,577.6,2023-11-03


In [25]:
short_trail_df

,Short Stop
Date,
2023-09-27,44766.10
2023-09-28,44766.10
2023-09-29,44766.10
2023-10-03,44766.10
2023-10-04,44766.10
2023-10-05,44766.10
2023-10-06,44360.60
2023-10-09,44360.60
2023-10-10,44360.60


In [26]:
long_trades_df=pd.DataFrame(long_trades)
long_trail_df = pd.DataFrame(long_trail) 

long_trail_df.set_index('Date', inplace=True)
long_trades_df.set_index('Entry Date', inplace=True)

In [27]:
long_trades_df

,Trade direction,Long Entry Price,Stop,Exit Price,Profit/Loss,Exit Date
Entry Date,,,,,,
2023-10-11,Long,44554.9,43886.50,43886.50,-668.40,2023-10-19
2023-11-03,Long,43318.3,43583.95,43583.95,265.65,2023-11-22
2023-11-24,Long,43769.1,47445.30,48292.25,4523.15,2023-12-29


In [28]:
long_trail_df

,Long Stop
Date,
2023-10-11,43886.50
2023-10-12,43886.50
2023-10-13,43886.50
2023-10-16,43886.50
2023-10-17,43886.50
2023-10-18,43886.50
2023-10-19,43886.50
2023-11-03,42700.95
2023-11-06,42700.95


In [29]:
dt = dt.merge(short_trail_df[['Short Stop']], left_index=True, right_index=True, how='left') \
          .merge(short_trades_df[['Short Entry Price']], left_index=True, right_index=True, how='left')

In [30]:
dt = dt.merge(long_trail_df[['Long Stop']], left_index=True, right_index=True, how='left') \
          .merge(long_trades_df[['Long Entry Price']], left_index=True, right_index=True, how='left')

In [33]:
dt[20:80]

,Open,High,Low,Close,Swing,Short Stop,Short Entry Price,Long Stop,Long Entry Price
2023-10-03,44561.50,44566.30,44243.10,44399.05,NaN,44766.10,NaN,NaN,NaN
2023-10-04,44108.15,44161.35,43857.50,43964.05,SL,44766.10,NaN,NaN,NaN
2023-10-05,44181.35,44391.85,44108.30,44213.35,NaN,44766.10,NaN,NaN,NaN
2023-10-06,44395.40,44500.70,44242.95,44360.60,SH,44360.60,NaN,NaN,NaN
2023-10-09,44057.80,44113.20,43796.75,43886.50,SL,44360.60,NaN,NaN,NaN
2023-10-10,44027.55,44487.25,44004.55,44360.15,NaN,44360.60,NaN,NaN,NaN
2023-10-11,44554.90,44710.55,44411.40,44516.90,NaN,44360.60,NaN,43886.50,44554.9
2023-10-12,44571.55,44693.05,44530.05,44599.20,SH,NaN,NaN,43886.50,NaN
2023-10-13,44322.05,44563.05,44203.70,44287.95,NaN,NaN,NaN,43886.50,NaN
2023-10-16,44204.75,44356.95,44044.40,44225.90,NaN,NaN,NaN,43886.50,NaN
